<a href="https://colab.research.google.com/github/kimujinu/Machine-learning/blob/main/%EC%B6%94%EC%B2%9C_%EC%8B%9C%EC%8A%A4%ED%85%9C(Recommender_System).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 추천 시스템 (Recommender Systems)

* 추천 시스템은 크게 두가지로 구분 가능
  * 컨텐츠 기반 필터링 (content-based filtering)
  * 협업 필터링 (collaborative filtering)
* 두가지를 조합한 hybrid 방식도 가능
* 컨텐츠 기반 필터링은 지금까지 사용자의 이전 행동과 명시적 피드백을 통해 사용자가 좋아하는 것과 유사한 항목을 추천
* 협업 필터링은 사용자와 항목간의 유사성을 동시에 사용해 추천

## Surprise

* 추천 시스템 개발을 위한 라이브러리
* 다양한 모델과 데이터 제공
* scikit-learn과 유사한 사용 방법

In [ ]:
!pip install scikit-surprise

     |████████████████████████████████| 11.8 MB 5.5 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1619380 sha256=7baeb54cd0d16b5643cda7c483725a978266e692998fb2484e03194f46c1595f
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


간단한 surprise 실습

In [ ]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

In [ ]:
data = Dataset.load_builtin('ml-100k',prompt=False)
data.raw_ratings[:10]

Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


[('196', '242', 3.0, '881250949'),
 ('186', '302', 3.0, '891717742'),
 ('22', '377', 1.0, '878887116'),
 ('244', '51', 2.0, '880606923'),
 ('166', '346', 1.0, '886397596'),
 ('298', '474', 4.0, '884182806'),
 ('115', '265', 2.0, '881171488'),
 ('253', '465', 5.0, '891628467'),
 ('305', '451', 3.0, '886324817'),
 ('6', '86', 3.0, '883603013')]

In [ ]:
model = SVD()

In [ ]:
cross_validate(model,data,measures=['rmse','mae'],cv=5,verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9375  0.9252  0.9310  0.9360  0.9432  0.9346  0.0061  
MAE (testset)     0.7378  0.7300  0.7336  0.7376  0.7432  0.7365  0.0044  
Fit time          4.84    4.78    4.78    4.75    4.77    4.78    0.03    
Test time         0.18    0.26    0.26    0.24    0.17    0.22    0.04    


{'fit_time': (4.8421103954315186,
  4.775526762008667,
  4.7849345207214355,
  4.750004768371582,
  4.7707600593566895),
 'test_mae': array([0.73780631, 0.73004767, 0.73359941, 0.7376223 , 0.74322587]),
 'test_rmse': array([0.93749988, 0.92523034, 0.9309571 , 0.93603609, 0.94319786]),
 'test_time': (0.17934560775756836,
  0.2638859748840332,
  0.2578146457672119,
  0.24410533905029297,
  0.17145180702209473)}

## 컨텐츠 기반 필터링 (Content-based Filtering)

* 컨텐츠 기반 필터링은 이전의 행동과 명시적 피드백을 통해 좋아하는 것과 유사한 항목을 추천
  * ex) 내가 지금 까지 시청한 영화 목록과 다른 사용자의 시청 목록을 비교해 나와 비슷한 취향의 사용자가 시청한 영화를 추천
* 유사도를 기반으로 추천
* 컨텐츠 기반 필터링은 다음과 같은 장단점이 있다.
  * 장점
    * 많은 수의 사용자를 대상으로 쉽게 확장 가능
    * 사용자가 관심을 갖지 않던 상품 추천 가능
  * 단점
    * 입력 특성을 직접 설계해야 하기 때문에 많은 도메인 지식이 필요
    * 사용자의 기존 관심사항을 기반으로만 추천 가능

In [ ]:
import numpy as np
from surprise import Dataset

* 이진 벡터의 내적을 통해 다른 사용자들과의 유사도 구하기
* 나와 가장 높은 유사도를 가진 사용자의 시청 목록을 추천

In [ ]:
data = Dataset.load_builtin('ml-100k',prompt=False)
raw_data = np.array(data.raw_ratings,dtype=int)

In [ ]:
raw_data[:,0] -= 1
raw_data[:,1] -= 1

In [ ]:
n_users = np.max(raw_data[:,0])
n_movies = np.max(raw_data[:,1])
shape = (n_users + 1, n_movies + 1)
shape

(943, 1682)

In [ ]:
adj_matrix = np.ndarray(shape,dtype=int)
for user_id,movie_id, rating, time in raw_data:
  adj_matrix[user_id][movie_id] = 1.
adj_matrix

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]])

In [ ]:
my_id, my_vector = 0,adj_matrix[0]
best_match, best_match_id, best_match_vector = -1,-1,[]

for user_id, user_vector in enumerate(adj_matrix):
  if my_id != user_id:
    similarity = np.dot(my_vector,user_vector)
    if similarity > best_match:
      best_match = similarity
      best_match_id = user_id
      best_match_vector = user_vector
    
print("Best Match : {} Best Match ID : {}". format(best_match,best_match_id))

Best Match : 183 Best Match ID : 275


In [ ]:
recommend_list = []
for i,log in enumerate(zip(my_vector,best_match_vector)):
  log1,log2 = log
  if log1 < 1. and log2 > 0:
    recommend_list.append(i)
print(recommend_list)

[272, 273, 275, 280, 281, 283, 287, 288, 289, 290, 292, 293, 297, 299, 300, 301, 302, 306, 312, 314, 315, 316, 317, 321, 322, 323, 324, 327, 330, 331, 332, 333, 339, 342, 345, 346, 353, 354, 355, 356, 357, 363, 364, 365, 366, 372, 374, 378, 379, 381, 382, 383, 384, 385, 386, 387, 390, 391, 392, 394, 395, 396, 398, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 412, 414, 416, 417, 418, 419, 420, 422, 424, 425, 426, 427, 428, 430, 431, 432, 435, 442, 446, 447, 448, 449, 450, 451, 452, 454, 455, 457, 460, 461, 462, 468, 469, 470, 471, 472, 473, 474, 478, 495, 500, 507, 517, 522, 525, 530, 539, 540, 543, 545, 546, 548, 549, 550, 551, 553, 557, 558, 560, 561, 562, 563, 565, 566, 567, 568, 570, 571, 574, 575, 576, 577, 580, 581, 582, 585, 587, 589, 590, 594, 596, 602, 623, 626, 627, 630, 633, 635, 639, 646, 648, 651, 652, 654, 657, 664, 668, 671, 677, 678, 681, 683, 684, 685, 690, 691, 692, 695, 696, 708, 709, 714, 718, 719, 720, 724, 726, 727, 731, 733, 734, 736, 738, 741, 742, 745,

* 유클리드 거리를 사용해 추천
$$euclidean = \sqrt{\sum_{d=1}^{D}(A_i - B_i)^2}$$
* 거리가 가까울 수록(값이 작을 수록) 나와 유사한 사용자

In [ ]:
my_id, my_vector = 0,adj_matrix[0]
best_match, best_match_id, best_match_vector = 9999,-1,[]

for user_id, user_vector in enumerate(adj_matrix):
  if my_id != user_id:
    euclidean_dist = np.sqrt(np.sum(np.square(my_vector-user_vector)))
    if euclidean_dist < best_match:
      best_match = euclidean_dist
      best_match_id = user_id
      best_match_vector = user_vector
    
print("Best Match : {} Best Match ID : {}". format(best_match,best_match_id))

Best Match : 14.832396974191326 Best Match ID : 737


In [ ]:
recommend_list = []
for i,log in enumerate(zip(my_vector,best_match_vector)):
  log1,log2 = log
  if log1 < 1. and log2 > 0:
    recommend_list.append(i)
print(recommend_list)

[297, 312, 317, 342, 356, 366, 379, 384, 392, 402, 404, 407, 417, 422, 428, 433, 448, 454, 469, 473, 495, 510, 516, 526, 527, 549, 567, 602, 635, 649, 650, 654, 658, 661, 664, 696, 731, 746, 750, 754, 915, 918, 925, 929, 950, 968, 1015, 1046]


* 코사인 유사도를 사용해 추천

\begin{equation}
cos \theta = \frac{A \cdot B}{||A|| \times ||B||}
\end{equation}
* 두 벡터가 이루고 있는 각을 계산

In [ ]:
def compute_cos_similarity(v1,v2):
  norm1 = np.sqrt(np.sum(np.square(v1)))
  norm2 = np.sqrt(np.sum(np.square(v2)))
  dot = np.dot(v1,v2)
  return dot / (norm1 * norm2)

In [ ]:
my_id, my_vector = 0,adj_matrix[0]
best_match, best_match_id, best_match_vector = -1,-1,[]

for user_id, user_vector in enumerate(adj_matrix):
  if my_id != user_id:
    cos_similarity = compute_cos_similarity(my_vector,user_vector)
    if cos_similarity > best_match:
      best_match = cos_similarity
      best_match_id = user_id
      best_match_vector = user_vector
    
print("Best Match : {} Best Match ID : {}". format(best_match,best_match_id))

Best Match : 0.5278586163659506 Best Match ID : 915


In [ ]:
recommend_list = []
for i,log in enumerate(zip(my_vector,best_match_vector)):
  log1,log2 = log
  if log1 < 1. and log2 > 0:
    recommend_list.append(i)
print(recommend_list)

[272, 275, 279, 280, 283, 285, 289, 294, 297, 316, 317, 355, 365, 366, 368, 379, 380, 381, 384, 386, 392, 398, 401, 404, 416, 420, 422, 424, 426, 427, 430, 432, 450, 460, 461, 466, 469, 471, 473, 474, 475, 479, 482, 483, 497, 505, 508, 510, 511, 522, 526, 527, 529, 530, 534, 536, 540, 545, 548, 549, 556, 557, 558, 560, 565, 567, 568, 569, 577, 580, 581, 582, 592, 596, 630, 635, 639, 641, 649, 651, 654, 673, 677, 678, 683, 684, 692, 696, 701, 703, 707, 708, 709, 712, 714, 719, 720, 726, 731, 734, 736, 738, 740, 745, 747, 754, 755, 761, 762, 763, 766, 780, 789, 791, 805, 819, 823, 824, 830, 843, 862, 865, 918, 929, 930, 938, 942, 943, 947, 958, 959, 960, 970, 977, 1004, 1008, 1009, 1010, 1013, 1041, 1045, 1069, 1072, 1073, 1078, 1097, 1100, 1108, 1112, 1118, 1134, 1193, 1205, 1207, 1216, 1219, 1267, 1334, 1400, 1427, 1596, 1681]


기존 방법에 명시적 피드백(사용자가 평가한 영화 점수)을 추가해 실험

In [ ]:
adj_matrix = np.ndarray(shape,dtype = int)
for user_id,movie_id,rating, time in raw_data:
  adj_matrix[user_id][movie_id] = rating
adj_matrix

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

In [ ]:
my_id, my_vector = 0,adj_matrix[0]
best_match, best_match_id, best_match_vector = 9999,-1,[]

for user_id, user_vector in enumerate(adj_matrix):
  if my_id != user_id:
    euclidean_dist = np.sqrt(np.sum(np.square(my_vector-user_vector)))
    if euclidean_dist < best_match:
      best_match = euclidean_dist
      best_match_id = user_id
      best_match_vector = user_vector
    
print("Best Match : {} Best Match ID : {}". format(best_match,best_match_id))

Best Match : 55.06359959174482 Best Match ID : 737


In [ ]:
my_id, my_vector = 0,adj_matrix[0]
best_match, best_match_id, best_match_vector = -1,-1,[]

for user_id, user_vector in enumerate(adj_matrix):
  if my_id != user_id:
    cos_similarity = compute_cos_similarity(my_vector,user_vector)
    if cos_similarity > best_match:
      best_match = cos_similarity
      best_match_id = user_id
      best_match_vector = user_vector
    
print("Best Match : {} Best Match ID : {}". format(best_match,best_match_id))

Best Match : 0.569065731527988 Best Match ID : 915


## 협업 필터링(Collaborative Filtering)

* 사용자와 항목의 유사성을 동시에 고려해 추천
* 기존에 내 관심사가 아닌 항목이라도 추천 가능
* 자동으로 임베딩 학습 가능
* 협업 필터링은 다음과 같은 장단점을 갖고 있다.
  * 장점
    * 자동으로 임베딩을 학습하기 때문에 도메인 지식이 필요 없다.
    * 기존의 관심사가 아니더라도 추천 가능
  * 단점
    * 학습 과정에 나오지 않은 항목은 임베딩을 만들 수 없음
    * 추가 특성을 사용하기 어려움

In [ ]:
from surprise import KNNBasic, SVD, SVDpp, NMF
from surprise import Dataset
from surprise.model_selection import cross_validate

In [ ]:
data = Dataset.load_builtin('ml-100k',prompt=False)

* KNN을 사용한 협업 필터링

In [ ]:
model = KNNBasic()
cross_validate(model, data, measures=['rmse','mae'],cv=5,n_jobs=4,verbose=True)

Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9761  0.9815  0.9772  0.9799  0.9785  0.9787  0.0019  
MAE (testset)     0.7693  0.7714  0.7723  0.7753  0.7751  0.7727  0.0023  
Fit time          0.46    0.80    0.79    0.66    0.67    0.68    0.12    
Test time         7.06    8.56    8.55    6.54    4.11    6.96    1.64    


{'fit_time': (0.45621657371520996,
  0.8021507263183594,
  0.7882912158966064,
  0.6629374027252197,
  0.6740493774414062),
 'test_mae': array([0.76929452, 0.77139662, 0.77234109, 0.77526982, 0.77512766]),
 'test_rmse': array([0.97611444, 0.98154992, 0.97724229, 0.97990908, 0.97853425]),
 'test_time': (7.062639951705933,
  8.563608407974243,
  8.551494836807251,
  6.5353333950042725,
  4.1073691844940186)}

* SVD를 사용한 협업 필터링

In [ ]:
model = SVD()
cross_validate(model, data, measures=['rmse','mae'],cv=5,n_jobs=4,verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9423  0.9365  0.9340  0.9333  0.9353  0.9363  0.0032  
MAE (testset)     0.7422  0.7394  0.7374  0.7369  0.7339  0.7380  0.0028  
Fit time          11.87   14.64   15.58   12.64   8.63    12.67   2.42    
Test time         0.61    0.59    0.58    0.32    0.17    0.45    0.18    


{'fit_time': (11.865983963012695,
  14.637927055358887,
  15.58346176147461,
  12.63832712173462,
  8.62897253036499),
 'test_mae': array([0.74222853, 0.73944586, 0.73742194, 0.73689544, 0.73390297]),
 'test_rmse': array([0.94232416, 0.93648142, 0.93398755, 0.93327375, 0.93525942]),
 'test_time': (0.609351634979248,
  0.5949349403381348,
  0.5825481414794922,
  0.31743741035461426,
  0.16983318328857422)}

* NMF를 사용한 협업 필터링

In [ ]:
model = NMF()
cross_validate(model, data, measures=['rmse','mae'],cv=5,n_jobs=4,verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9595  0.9695  0.9595  0.9713  0.9679  0.9655  0.0051  
MAE (testset)     0.7544  0.7630  0.7564  0.7640  0.7596  0.7595  0.0037  
Fit time          12.89   16.58   16.32   13.74   9.24    13.75   2.67    
Test time         0.64    0.52    0.41    0.24    0.15    0.39    0.18    


{'fit_time': (12.890738248825073,
  16.575007438659668,
  16.321446418762207,
  13.738579750061035,
  9.236399412155151),
 'test_mae': array([0.75443071, 0.76303007, 0.75643176, 0.76395691, 0.75959184]),
 'test_rmse': array([0.95952683, 0.96950364, 0.95947117, 0.97129861, 0.96791867]),
 'test_time': (0.6438038349151611,
  0.5221657752990723,
  0.4077279567718506,
  0.24451875686645508,
  0.15354704856872559)}

* SVD++를 사용한 협업 필터링

In [ ]:
model = SVDpp()
cross_validate(model, data, measures=['rmse','mae'],cv=5,n_jobs=4,verbose=True)
# 너무 오래 걸려... 비효율적..

## 하이브리드(Hybrid)

* 컨텐츠 기반 필터링과 협업 필터링을 조합한 방식
* 많은 하이브리드 방식이 존재
* 실습에서는 협업 필터링으로 임베딩을 학습하고 컨텐츠 기반 필터링으로 유사도 기반 추천을 수행하는 추천 엔진 개발

In [ ]:
import numpy as np
from sklearn.decomposition import randomized_svd, non_negative_factorization
from surprise import Dataset

In [ ]:
data = Dataset.load_builtin('ml-100k', prompt=False)
raw_data = np.array(data.raw_ratings, dtype=int)
raw_data[:, 0] -= 1
raw_data[:, 1] -= 1


In [ ]:
n_users = np.max(raw_data[:, 0])
n_movies = np.max(raw_data[:, 1])
shape = (n_users + 1, n_movies + 1)
shape


(943, 1682)

In [ ]:
adj_matrix = np.ndarray(shape, dtype=int)
for user_id, movie_id, rating, time in raw_data:
  adj_matrix[user_id][movie_id] = rating

In [ ]:
adj_matrix

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

In [ ]:
U, S, V = randomized_svd(adj_matrix, n_components=2)
S = np.diag(S)


/usr/local/lib/python3.7/dist-packages/sklearn/utils/extmath.py:376: FutureWarning: If 'random_state' is not supplied, the current default is to use 0 as a fixed seed. This will change to  None in version 1.2 leading to non-deterministic results that better reflect nature of the randomized_svd solver. If you want to silence this warning, set 'random_state' to an integer seed or to None explicitly depending if you want your code to be deterministic or not.
  FutureWarning,


In [ ]:
print(U.shape)
print(S.shape)
print(V.shape)


(943, 2)
(2, 2)
(2, 1682)


In [ ]:
np.matmul(np.matmul(U, S), V)


array([[ 5.16407464e+02,  3.25726485e+02,  1.52060722e+02, ...,
        -5.22362777e+01, -7.84835374e+01, -1.12546508e+02],
       [-3.50622861e+02, -2.21090909e+02, -1.03150208e+02, ...,
         3.55982504e+01,  5.34141566e+01,  7.65463011e+01],
       [-1.80373055e+02, -1.13724098e+02, -5.30456933e+01, ...,
         1.83389850e+01,  2.75031286e+01,  3.94040385e+01],
       ...,
       [ 2.00303757e-01,  1.49731645e-01,  9.21430761e-02, ...,
         2.62812806e-02,  1.43048360e-02,  2.70569238e-03],
       [ 6.88116663e-01,  5.33620204e-01,  3.43820399e-01, ...,
         1.28566568e-01,  8.59461793e-02,  4.74256969e-02],
       [ 1.67117098e+00,  1.22435947e+00,  7.33492113e-01, ...,
         1.69761863e-01,  7.17502603e-02, -2.67394679e-02]])

* 사용자 기반 추천
* 나와 비슷한 취향을 가진 다른 사용자의 행동을 추천
* 사용자 특징 벡터의 유사도 사용

In [ ]:
my_id, my_vector = 0, U[0]
best_match, best_match_id, best_match_vector = -1, -1, []
for user_id, user_vector in enumerate(U):
  if my_id != user_id:
    cos_similarity = compute_cos_similarity(my_vector, user_vector)
    if cos_similarity > best_match:
      best_match = cos_similarity
      best_match_id = user_id
      best_match_vector = user_vector

print('Best Match: {}, Best Match ID: {}'.format(best_match, best_match_id))

Best Match: 0.9999998577761066, Best Match ID: 7


In [ ]:
recommend_list = []
for i, log in enumerate(zip(adj_matrix[my_id], adj_matrix[best_match_id])):
  log1, log2 = log
  if log1 < 1. and log2 > 0.:
    recommend_list.append(i)
print(recommend_list)


[272, 293, 300, 335, 337, 340, 357, 384, 402, 430, 434, 456, 509, 510, 517, 549, 565, 567, 650, 683, 684, 685, 686, 687, 688]


* 항목 기반 추천
* 내가 본 항목과 비슷한 항목을 추천
* 항목 특징 벡터의 유사도 사용

In [ ]:
my_id, my_vector = 0, V.T[0]
best_match, best_match_id, best_match_vector = -1, -1, []
for user_id, user_vector in enumerate(V.T):
  if my_id != user_id:
    cos_similarity = compute_cos_similarity(my_vector, user_vector)
    if cos_similarity > best_match:
      best_match = cos_similarity
      best_match_id = user_id
      best_match_vector = user_vector
print('Best Match: {}, Best Match ID: {}'.format(best_match, best_match_id))

Best Match: 0.9999963669411721, Best Match ID: 68


In [ ]:
recommend_list = []
for i, user_vector in enumerate(adj_matrix):
  if adj_matrix[i][my_id] > 0.9:
    recommend_list.append(i)
print(recommend_list)


[0, 1, 4, 5, 9, 12, 14, 15, 16, 17, 19, 20, 22, 24, 25, 37, 40, 41, 42, 43, 44, 45, 46, 47, 48, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 76, 78, 79, 80, 81, 82, 83, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 116, 117, 118, 119, 120, 121, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 187, 188, 189, 190, 191, 192, 193, 194, 195, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 212, 214, 215, 216, 217, 219, 221, 222, 223, 224, 225, 226, 227, 229, 230, 231, 233, 234, 235, 238, 239, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 2

* 비음수 행렬 분해를 사용한 하이브리드 추천

In [ ]:
adj_matrix

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

In [ ]:
A, B, iter = non_negative_factorization(adj_matrix, n_components=2)

In [ ]:
np.matmul(A, B)

* 사용자 기반 추천

In [ ]:
my_id, my_vector = 0, U[0]  # 내 id가 0번 이라고 가정
b

In [ ]:
recommend_list = []
for i, log in enumerate(zip(adj_matrix[my_id], adj_matrix[best_match_id])):
  log1, log2 = log
  if log1 < 1. and log2 > 0.:
    recommend_list.append(i)
print(recommend_list)

[272, 273, 274, 275, 276, 277, 278, 280, 281, 282, 283, 285, 286, 287, 288, 290, 291, 292, 293, 294, 295, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 319, 320, 321, 322, 323, 324, 326, 328, 329, 330, 331, 332, 333, 334, 335, 336, 338, 340, 341, 343, 344, 345, 346, 347, 348, 349, 350, 351, 353, 354, 355, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 369, 370, 371, 372, 374, 375, 376, 377, 378, 379, 381, 383, 384, 385, 386, 387, 388, 389, 390, 391, 393, 394, 395, 396, 397, 398, 400, 401, 402, 403, 404, 405, 407, 408, 409, 410, 411, 412, 414, 415, 416, 417, 419, 421, 422, 423, 424, 426, 427, 428, 429, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 448, 449, 450, 451, 452, 453, 455, 457, 458, 459, 462, 463, 464, 465, 467, 468, 469, 470, 472, 474, 475, 476, 477, 478, 479, 480, 481, 482, 484, 485, 486, 487, 489, 490, 491, 492, 493, 494, 495, 496, 497, 498, 499, 501, 502, 503, 504, 505, 506,

* 항목 기반 추천

In [ ]:
my_id, my_vector = 0, B.T[0]  # 기준 항목이 0번 이라고 가정
best_match, best_match_id, best_match_vector = -1, -1, []

for item_id, item_vector in enumerate(V.T):
  if my_id != item_id:
    cos_similarity = compute_cos_similarity(my_vector, item_vector)
    if cos_similarity > best_match:
      best_match = cos_similarity
      best_match_id = item_id
      best_match_vector = item_vector

print('Best Match: {}, Best Match ID: {}'.format(best_match, best_match_id))

In [ ]:
recommend_list = []
for i, user_vector in enumerate(adj_matrix):
  if adj_matrix[i][my_id] > 0.9:
    recommend_list.append(i)
print(recommend_list)

[0, 1, 4, 5, 9, 12, 14, 15, 16, 17, 19, 20, 22, 24, 25, 37, 40, 41, 42, 43, 44, 45, 46, 47, 48, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 76, 78, 79, 80, 81, 82, 83, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 116, 117, 118, 119, 120, 121, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 187, 188, 189, 190, 191, 192, 193, 194, 195, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 212, 214, 215, 216, 217, 219, 221, 222, 223, 224, 225, 226, 227, 229, 230, 231, 233, 234, 235, 238, 239, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 2